### 处理数据

In [12]:
import pandas as pd

csv_path = r"D:\article\SynologyDrive\LAI-LST-Asymmetric\data\outCSV\01_matching_results\Boreal\Winter_mode1.csv"

df = pd.read_csv(csv_path, low_memory=False)
# df.columns.tolist()
df['lst_daily_diff_lai_1'] = (df['lst_day_diff_lai_1']+df['lst_night_diff_lai_1']) / 2
df['lst_daily_diff_lai_2'] = (df['lst_day_diff_lai_2']+df['lst_night_diff_lai_2']) / 2

df.to_csv(csv_path, index=False)

In [14]:
import pandas as pd

csv_path = r"D:\article\SynologyDrive\LAI-LST-Asymmetric\data\outCSV\01_matching_results\Boreal\Winter_mode1.csv"

df = pd.read_csv(csv_path, low_memory=False)
# df.columns.tolist()
df['hg_diff_lai_1'] = -df['sw_diff_lai_1'] - df['lw_diff_lai_1'] - df['le_diff_lai_1']
df['hg_diff_lai_2'] = -df['sw_diff_lai_2'] - df['lw_diff_lai_2'] - df['le_diff_lai_2']

df.to_csv(csv_path, index=False)

In [10]:
import numpy as np
import pandas as pd

# 1. 读取CSV文件
csv_path = r'D:\article\SynologyDrive\LAI-LST-Asymmetric\data\outCSV\01_matching_results\Boreal\Winter_mode1.csv'

df = pd.read_csv(csv_path)

print(f"✓ CSV文件读取成功")
print(f"数据行数: {len(df)}")
print(f"数据列数: {len(df.columns)}")

# 2. 查看grid_type的分布情况
print("\n=== Grid Type 分布情况 ===")
print("\ngrid_type_1的值分布:")
print(df['grid_type_1'].value_counts().sort_index())
print("\ngrid_type_2的值分布:")
print(df['grid_type_2'].value_counts().sort_index())

# 查看LAI变化区间分布(如果需要的话)
if 'rate_range_1' in df.columns:
    print("\n=== LAI变化区间分布 ===")
    print("\nrate_range_1分布:")
    print(df['rate_range_1'].value_counts())
    print("\nrate_range_2分布:")
    print(df['rate_range_2'].value_counts())

C:\Users\liupa\AppData\Local\Temp\ipykernel_4616\477943330.py:7: DtypeWarning: Columns (162,163) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✓ CSV文件读取成功
数据行数: 703120
数据列数: 190

=== Grid Type 分布情况 ===

grid_type_1的值分布:
grid_type_1
1    703120
Name: count, dtype: int64

grid_type_2的值分布:
grid_type_2
-1    703120
Name: count, dtype: int64

=== LAI变化区间分布 ===

rate_range_1分布:
rate_range_1
[0.0, 0.2)    644543
[0.2, 0.4)     56460
[0.4, 0.6)      2092
[0.6, 0.8)        23
[0.8, +∞)          1
Name: count, dtype: int64

rate_range_2分布:
rate_range_2
[0.0, 0.2)    644543
[0.2, 0.4)     56460
[0.4, 0.6)      2092
[0.6, 0.8)        23
[0.8, +∞)          1
Name: count, dtype: int64


In [ ]:
# 3. 选择要操作的LAI变化类型
# 选择 1 或 2，表示操作 _1 或 _2 后缀的列
target_suffix = 2  # 修改这里: 1表示操作_1后缀的列, 2表示操作_2后缀的列
var = "LST_DAY"

print(f"\n选择操作后缀为 _{target_suffix} 的{var}列")

# 4. 根据grid_type筛选要操作的数据行(可选)
# 如果需要只对特定grid_type的行进行操作，取消下面的注释
use_grid_type_filter = True  # 改为True启用grid_type筛选
target_grid_type = -1  # 选择要操作的grid_type值

if use_grid_type_filter:
    if target_suffix == 1:
        filtered_df = df[df['grid_type_1'] == target_grid_type]
    else:
        filtered_df = df[df['grid_type_2'] == target_grid_type]
    print(f"筛选 grid_type_{target_suffix}={target_grid_type} 后的数据行数: {len(filtered_df)}")
else:
    filtered_df = df  # 不筛选，对所有行操作
    print(f"对所有 {len(df)} 行数据进行操作")

In [ ]:
# 5. 进一步根据LAI变化区间筛选(可选)
use_rate_range_filter = False  # 是否使用LAI变化区间筛选，改为True启用

if use_rate_range_filter and f'rate_range_{target_suffix}' in df.columns:
    target_rate_range = '[0.6, 0.8)'  # 修改这里选择目标区间
    filtered_df = filtered_df[filtered_df[f'rate_range_{target_suffix}'] == target_rate_range]
    print(f"进一步筛选 rate_range_{target_suffix}={target_rate_range} 后的数据行数: {len(filtered_df)}")

In [ ]:
# 6. 找出指定后缀的数据列
le_columns = [col for col in df.columns
              if f'{var}' in col.upper()
              and 'STD' not in col.upper()
              and col.endswith(f'_{target_suffix}')]

print(f"\n=== 找到的{var}相关列 (后缀为_{target_suffix}, 共{len(le_columns)}个) ===")
for i, col in enumerate(le_columns, 1):
    print(f"{i}. {col}")

# 7. 显示操作前的示例数据
if len(filtered_df) > 0 and len(le_columns) > 0:
    print(f"\n=== 操作前示例数据 ===")
    print(filtered_df[le_columns].head(3))

In [ ]:
# 8. 对筛选出的数据行进行操作
for col in le_columns:
    mask = filtered_df.index
    # 只对非NaN的值进行操作
    non_nan_mask = df.loc[mask, col].notna()
    df.loc[mask[non_nan_mask], col] = df.loc[mask[non_nan_mask], col]/2

print(f"\n✓ 已对 {len(filtered_df)} 行数据的 {len(le_columns)} 个{var}列(后缀_{target_suffix})进行处理")

# 9. 显示操作后的示例数据
if len(filtered_df) > 0 and len(le_columns) > 0:
    print(f"\n=== 操作后示例数据 ===")
    print(df.loc[filtered_df.index, le_columns].head(3))

# 10. 保存文件
df.to_csv(csv_path, index=False)
print(f"\n✓ 数据已保存回原始CSV文件")

### 计算能量平衡参数

In [ ]:
import pandas as pd
import numpy as np

# 读取数据
csv_path = r"D:\article\01_matching_results\Boreal\Summer_mode1.csv"
df = pd.read_csv(csv_path)

# 需要的数据列
delta_albedo_1 = df['albedo_diff_lai_2']
delta_albedo_2 = df['albedo_diff_lai_1']
delta_albedo_1_std = df['albedo_diff_lai_std_2']
delta_albedo_2_std = df['albedo_diff_lai_std_1']

sw_downward_1 = df['DSR_Begin_mean_2']
sw_downward_2 = df['DSR_Begin_mean_1']
sw_downward_1_std = df['DSR_Begin_std_2']
sw_downward_2_std = df['DSR_Begin_std_1']

delta_le_1 = df['le_diff_lai_2']
delta_le_2 = df['le_diff_lai_1']
delta_le_1_std = df['le_diff_lai_std_2']
delta_le_2_std = df['le_diff_lai_std_1']

delta_lst_daily_1 = df['lst_daily_diff_lai_2']
delta_lst_daily_2 = df['lst_daily_diff_lai_1']
delta_lst_daily_1_std = df['lst_daily_diff_lai_std_2']
delta_lst_daily_2_std = df['lst_daily_diff_lai_std_1']
lst_daily_begin_1 = df['LST_Daily_Begin_mean_2'] + 273.15
lst_daily_begin_2 = df['LST_Daily_Begin_mean_1'] + 273.15
lst_daily_begin_1_std = df['LST_Daily_Begin_std_2'] + 273.15
lst_daily_begin_2_std = df['LST_Daily_Begin_std_1'] + 273.15

# 常量定义
# ϵ(单位: nm)
# σ(单位: W(m−2 K−4))
rng = np.random.default_rng(96)

epsilon_29 = np.arange(8400, 8700)
epsilon_31 = np.arange(10780, 11280)
epsilon_32 = np.arange(11770, 12270)

sample_29 = rng.choice(epsilon_29) / 10000
sample_31 = rng.choice(epsilon_31) / 10000
sample_32 = rng.choice(epsilon_32) / 10000

epsilon = 0.2122 * sample_29 + 0.3859 * sample_31 + 0.4029 * sample_32
# epsilon = epsilon_array.mean()

sigma = 5.67e-8

##### $\Delta SW_{\uparrow,LAI} = \Delta \text{albedo} \times SW_{\downarrow}$

In [ ]:
delta_sw_upward_lai_1 =  delta_albedo_1 * sw_downward_1
delta_sw_upward_lai_2 =  delta_albedo_2 * sw_downward_2

delta_sw_upward_lai_1_std = delta_albedo_1_std * sw_downward_1_std
delta_sw_upward_lai_2_std = delta_albedo_2_std * sw_downward_2_std

##### $\Delta LW_{\uparrow,LAI} = \epsilon \Delta LST_{LAI} \cdot 4\sigma LST^3$

In [ ]:
delta_lw_upward_lai_1 = epsilon * delta_lst_daily_1 * 4 * sigma * lst_daily_begin_1 ** 3
delta_lw_upward_lai_2 = epsilon * delta_lst_daily_2 * 4 * sigma * lst_daily_begin_2 ** 3

delta_lw_upward_lai_1_std = epsilon * delta_lst_daily_1_std * 4 * sigma * lst_daily_begin_1_std ** 3
delta_lw_upward_lai_2_std = epsilon * delta_lst_daily_2_std * 4 * sigma * lst_daily_begin_2_std ** 3

##### $\Delta(H + G)_{LAI} = -\Delta SW_{\uparrow,LAI} - \Delta LW_{\uparrow,LAI} - \Delta LE_{LAI}$

In [ ]:
delta_hg_lai_1 = -delta_sw_upward_lai_1 - delta_lw_upward_lai_1 - delta_le_1
delta_hg_lai_2 = -delta_sw_upward_lai_2 - delta_lw_upward_lai_2 - delta_le_2

delta_hg_lai_1_std = -delta_sw_upward_lai_1_std - delta_lw_upward_lai_1_std - delta_le_1_std
delta_hg_lai_2_std = -delta_sw_upward_lai_2_std - delta_lw_upward_lai_2_std - delta_le_2_std

In [ ]:
# 将结果写入原始数据中
# H+G
df['hg_diff_lai_1'] = delta_hg_lai_1
df['hg_diff_lai_2'] = delta_hg_lai_2

df['hg_diff_lai_1_std'] = delta_hg_lai_1_std
df['hg_diff_lai_2_std'] = delta_hg_lai_2_std

# SW_upward
df['sw_diff_lai_1'] = delta_sw_upward_lai_1
df['sw_diff_lai_2'] = delta_sw_upward_lai_2

df['sw_diff_lai_1_std'] = delta_sw_upward_lai_1_std
df['sw_diff_lai_2_std'] = delta_sw_upward_lai_2_std

# LW_upward
df['lw_diff_lai_1'] = delta_lw_upward_lai_1
df['lw_diff_lai_2'] = delta_lw_upward_lai_2

df['lw_diff_lai_1_std'] = delta_lw_upward_lai_1_std
df['lw_diff_lai_2_std'] = delta_lw_upward_lai_2_std

df.to_csv(csv_path, index=False)

In [ ]:
df['hg_diff_lai_1'] = -df['sw_diff_lai_1'] - df['lw_diff_lai_1'] - delta_le_1
df['hg_diff_lai_2'] = -df['sw_diff_lai_2'] - df['lw_diff_lai_2'] - delta_le_2

df.to_csv(csv_path, index=False)